In [0]:
res_json_data = spark.read.format('json').option('header', 'true').option('inferSchema', 'true').option('mode', 'permissive')\
    .load('/FileStore/tables/resturant_json_data.json')

res_json_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
%fs
ls /FileStore/tables/resturant_json_data.json

path,name,size,modificationTime
dbfs:/FileStore/tables/resturant_json_data.json,resturant_json_data.json,669503,1745497396000


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

ex_data = res_json_data.select('*', explode('restaurants').alias('new_added_restaurants'))  # converted array in struct
ex_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
drop_data = ex_data.drop('restaurants')
drop_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_added_restaurants: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- h

In [0]:
select_id_data = drop_data.select('new_added_restaurants.restaurant.R.res_id')
select_id_data.show()

+--------+
|  res_id|
+--------+
|17066603|
|17059541|
|17064405|
|17057797|
|17057591|
|17064266|
|17060516|
|17060320|
|17059060|
|17059012|
|17060869|
|17061231|
|17058534|
|17057925|
|17064031|
|17061237|
|17061253|
|17061296|
|17061205|
|17057397|
+--------+
only showing top 20 rows



In [0]:
est_data = drop_data.select('*', 'new_added_restaurants.restaurant.R.res_id', 'new_added_restaurants.restaurant.name', explode_outer('new_added_restaurants.restaurant.establishment_types').alias("new_added_e_types")).drop('new_added_restaurants.restaurant.establishment_types').drop('new_added_restaurants', 'message', 'results_found')
est_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- res_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- new_added_e_types: string (nullable = true)



In [0]:
est_data = drop_data.select('*', 'new_added_restaurants.restaurant.R.res_id', 'new_added_restaurants.restaurant.name', explode_outer('new_added_restaurants.restaurant.establishment_types').alias("new_added_e_types")).drop('new_added_restaurants.restaurant.establishment_types').drop('new_added_restaurants', 'message', 'results_found')
est_data.show()

+----+-------------+-------------+------+--------+--------------------+-----------------+
|code|results_shown|results_start|status|  res_id|                name|new_added_e_types|
+----+-------------+-------------+------+--------+--------------------+-----------------+
|null|           20|            1|  null|17066603|            The Coop|             null|
|null|           20|            1|  null|17059541|Maggiano's Little...|             null|
|null|           20|            1|  null|17064405|Tako Cheena by Po...|             null|
|null|           20|            1|  null|17057797|Bosphorous Turkis...|             null|
|null|           20|            1|  null|17057591|Bahama Breeze Isl...|             null|
|null|           20|            1|  null|17064266|Hawkers Asian Str...|             null|
|null|           20|            1|  null|17060516|Seasons 52 Fresh ...|             null|
|null|           20|            1|  null|17060320|Raglan Road Irish...|             null|
|null|    

In [0]:
# scd2

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

customer_dim_data = [

(1,'manish','arwal','india','N','2022-09-15','2022-09-25'),
(2,'vikash','patna','india','Y','2023-08-12',None),
(3,'nikita','delhi','india','Y','2023-09-10',None),
(4,'rakesh','jaipur','india','Y','2023-06-10',None),
(5,'ayush','NY','USA','Y','2023-06-10',None),
(1,'manish','gurgaon','india','Y','2022-09-25',None),
]

customer_schema= ['id','name','city','country','active','effective_start_date','effective_end_date']

customer_dim_df = spark.createDataFrame(data= customer_dim_data,schema=customer_schema)

sales_data = [

(1,1,'manish','2023-01-16','gurgaon','india',380),
(77,1,'manish','2023-03-11','bangalore','india',300),
(12,3,'nikita','2023-09-20','delhi','india',127),
(54,4,'rakesh','2023-08-10','jaipur','india',321),
(65,5,'ayush','2023-09-07','mosco','russia',765),
(89,6,'rajat','2023-08-10','jaipur','india',321)
]

sales_schema = ['sales_id', 'customer_id','customer_name', 'sales_date', 'food_delivery_address','food_delivery_country', 'food_cost']

sales_df = spark.createDataFrame(data=sales_data,schema=sales_schema)

In [0]:
# join 2 df to identify change in address

left_df = customer_dim_df.join(sales_df, customer_dim_df['id']==sales_df['customer_id'], 'left')
display(left_df)

id,name,city,country,active,effective_start_date,effective_end_date,sales_id,customer_id,customer_name,sales_date,food_delivery_address,food_delivery_country,food_cost
1,manish,arwal,india,N,2022-09-15,2022-09-25,77,1,manish,2023-03-11,bangalore,india,300
1,manish,arwal,india,N,2022-09-15,2022-09-25,1,1,manish,2023-01-16,gurgaon,india,380
2,vikash,patna,india,Y,2023-08-12,null,null,null,null,null,null,null,null
3,nikita,delhi,india,Y,2023-09-10,null,12,3,nikita,2023-09-20,delhi,india,127
4,rakesh,jaipur,india,Y,2023-06-10,null,54,4,rakesh,2023-08-10,jaipur,india,321
5,ayush,NY,USA,Y,2023-06-10,null,65,5,ayush,2023-09-07,mosco,russia,765
1,manish,gurgaon,india,Y,2022-09-25,null,77,1,manish,2023-03-11,bangalore,india,300
1,manish,gurgaon,india,Y,2022-09-25,null,1,1,manish,2023-01-16,gurgaon,india,380


In [0]:
# want only whose address is changed
#replace effective_start_date column as sales_date
# food_delivery_address as city

new_records = left_df.where((col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
        .withColumn('effective_start_date', col('sales_date'))\
            .select('customer_id', 'customer_name', col('food_delivery_address').alias('city'), 'food_delivery_country', 'active', 'effective_start_date', 'effective_end_date')
new_records.show()

+-----------+-------------+---------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|     city|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+---------+---------------------+------+--------------------+------------------+
|          1|       manish|bangalore|                india|     Y|          2023-03-11|              null|
|          5|        ayush|    mosco|               russia|     Y|          2023-09-07|              null|
+-----------+-------------+---------+---------------------+------+--------------------+------------------+



In [0]:
# now update their old records as inactive

old_records = left_df.where((col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
    .withColumn('active', lit('N'))\
        .withColumn('effective_end_date', col('sales_date'))\
            .select('customer_id', 'customer_name', 'city', 'food_delivery_country', 'active', 'effective_start_date', 'effective_end_date')
old_records.show()

+-----------+-------------+-------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|   city|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+-------+---------------------+------+--------------------+------------------+
|          1|       manish|gurgaon|                india|     N|          2022-09-25|        2023-03-11|
|          5|        ayush|     NY|               russia|     N|          2023-06-10|        2023-09-07|
+-----------+-------------+-------+---------------------+------+--------------------+------------------+



In [0]:
# find new customers and insert them

new_customers = sales_df.join(customer_dim_df, sales_df['customer_id'] == customer_dim_df['id'], 'leftanti')\
    .withColumn('active', lit('Y'))\
        .withColumn('effective_end_date', lit(None))\
            .withColumn('effective_start_date', col('sales_date'))\
                .select('customer_id', 'customer_name', 'food_delivery_address', 'food_delivery_country', 'active', 'effective_start_date', 'effective_end_date')
new_customers.show()

+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|food_delivery_address|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+
|          6|        rajat|               jaipur|                india|     Y|          2023-08-10|              null|
+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+



In [0]:
# merge all records in 1 df

final_records = customer_dim_df.union(new_records).union(old_records).union(new_customers)
final_records.show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|       NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|  gurgaon|  india|     Y|          2022-09-25|              null|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|       NY| russia|     N|          2023-06-10|      

In [0]:
# manish is in banglore and gurgaon also  => not fully complete

window = Window.partitionBy('id', 'active').orderBy(col('effective_start_date').desc())

final_records.withColumn('rankk', rank().over(window))